In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras
from tensorflow.keras.utils import to_categorical

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
ROOT_DIR = '../input/data/'
TRAIN_DIR = ROOT_DIR + 'imgs/train/'
TEST_DIR = ROOT_DIR + 'imgs/test/'
driver_imgs_list = pd.read_csv(ROOT_DIR + "driver_imgs_list.csv")
random_list = np.random.permutation(len(driver_imgs_list))[:1250]
df_copy = driver_imgs_list.iloc[random_list]
image_paths = [TRAIN_DIR+row.classname+'/'+row.img 
                   for (index, row) in df_copy.iterrows()]
label_list = [int(row.classname[1]) for (index, row) in df_copy.iterrows()]

In [ ]:
# One hot vector representation of labels
# y_labels_one_hot = to_categorical(label_list, dtype=np.int8)
x_img_path = np.array(image_paths)

dataset = []
for i in range(len(x_img_path)): # len(x_img_path)
    img = mpimg.imread(x_img_path[i]) 
    dataset.append([img,label_list[i]])
dataset = np.transpose(dataset)

In [ ]:
data = np.array(dataset[:][0])
IMG_HEIGHT = 240
IMG_WIDTH = 320

# data = data.reshape(-1,3,IMG_HEIGHT,IMG_WIDTH).transpose([0,2,3,1])
plt.imshow(data[0])

In [ ]:
data = np.array(dataset[:][0])
IMG_HEIGHT = 240
IMG_WIDTH = 320

# data = data.reshape(-1,3,IMG_HEIGHT,IMG_WIDTH).transpose([0,2,3,1])
plt.imshow(data[2])

In [ ]:
data_gray = [ color.rgb2gray(i) for i in data]
plt.imshow(data_gray[0])

In [ ]:
ppc = 16
hog_images = []
hog_features = []
for image in data_gray:
    fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    hog_images.append(hog_image)
    hog_features.append(fd)

In [ ]:
import pickle

## pickle hog_images

In [ ]:
with open('./hog_images.pt','wb') as f:
    pickle.dump(hog_images,f)

In [ ]:
with open('./hog_features.pt','wb') as f:
    pickle.dump(hog_features,f)

In [ ]:
plt.imshow(hog_images[10],cmap=plt.get_cmap('gray'))

In [ ]:
with open('./hog_features.pt','rb') as f:
    hog_features=pickle.load(f)

In [ ]:
with open('./hog_images.pt','rb') as f:
    hog_images=pickle.load(f)

In [ ]:
labels = np.array(dataset[:][1])

In [ ]:
clf = svm.SVC(probability=True)
hog_features = np.array(hog_features)
data_frame = []
for i in range(len(hog_features)):
    data_frame.append(np.hstack((hog_features[i],labels[i]))) 

np.random.shuffle(data_frame)

In [ ]:
percentage = 80
partition = int(len(hog_features)*percentage/100)

In [ ]:
data_frame = np.array(data_frame)
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
clf.fit(x_train,y_train)

In [ ]:
clf

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

In [ ]:
probs=clf.predict_proba(x_test)